<a href="https://colab.research.google.com/github/DSLab-Sutech/DSLab-Sutech/blob/main/ViolenceDetection_NoInformationLeak.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from google.colab import drive
import IPython.display as ipd
from matplotlib import pyplot as plt
import os
import librosa
from tensorflow.keras.utils import to_categorical
from keras import layers
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten,BatchNormalization
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D,SeparableConv2D
from keras.layers.convolutional import Conv1D,MaxPooling1D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import TimeDistributed
from keras.layers import LSTM
from sklearn import preprocessing
from keras import Model
from keras.metrics import Precision,Recall,Accuracy
import gc
from sklearn.model_selection import train_test_split


In [2]:
#connect colab to google drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
# def print_plot_play(x, Fs, text=''):
#     print('%s Fs = %d, x.shape = %s, x.dtype = %s' % (text, Fs, x.shape, x.dtype))
#     plt.figure(figsize=(8, 2))
#     plt.plot(x, color='gray')
#     plt.xlim([0, x.shape[0]])
#     plt.xlabel('Time (samples)')
#     plt.ylabel('Amplitude')
#     plt.tight_layout()
#     plt.show()
#     ipd.display(ipd.Audio(data=x, rate=Fs))

In [4]:
#load voices
Data=[]
Target=[]
Data_test = []
Target_test = []
sample_rate = 16000          # google say : We recommend a sample rate of at least 16 kHz in the audio files that you use for transcription with Speech-to-Text.
slot_length = 10          # each audio slot is slot_length seconds
number_of_samples = slot_length * sample_rate
overlap = 3       # samples have 'overlap' seconds of overlapping
step = overlap * sample_rate
for i in range(1,73):
  path=r'/content/drive/My Drive/Audio/Nonviolence/Normal' + str(i) + '.mp3'
  x, Fs = librosa.load(path, sr=sample_rate)

  for j in range(0,len(x),step):
    if(j+number_of_samples < len(x)):
      Data.append(x[j:j+number_of_samples])
      Target.append(0)

for i in range(73,91):
  path=r'/content/drive/My Drive/Audio/Nonviolence/Normal' + str(i) + '.mp3'
  x, Fs = librosa.load(path, sr=sample_rate)

  for j in range(0,len(x),step):
    if(j+number_of_samples < len(x)):
      Data_test.append(x[j:j+number_of_samples])
      Target_test.append(0)


In [5]:
#print_plot_play(Data[202], Fs = sample_rate)

In [6]:
for i in range(1,36):
  if i==15 or i==16 or i==17:
    continue
  path=r'/content/drive/My Drive/Audio/Violence/Violence' + str(i) + '.mp3'
  x, Fs = librosa.load(path, sr=None)
  for j in range(0,len(x),step):
    if(j+number_of_samples < len(x)):
      Data.append(x[j:j+number_of_samples])
      Target.append(1)

for i in range(36,45):
  if i==15 or i==16 or i==17:
    continue
  path=r'/content/drive/My Drive/Audio/Violence/Violence' + str(i) + '.mp3'
  x, Fs = librosa.load(path, sr=None)
  for j in range(0,len(x),step):
    if(j+number_of_samples < len(x)):
      Data_test.append(x[j:j+number_of_samples])
      Target_test.append(1)


In [7]:
len(Target_test)
#normalize data
#normal_data = preprocessing.normalize(Data)

578

In [8]:
#convert Data to Tensor
#height = width = 600
#X = np.asarray(Data).reshape(len(Data),height,width,1)
X = np.asarray(Data).reshape(len(Data),number_of_samples,1)
y = to_categorical(Target,num_classes=2)
#y = np.asarray(Target).astype('float32')
X_test = np.asarray(Data_test).reshape(len(Data_test),number_of_samples,1)
y_test = to_categorical(Target_test,num_classes=2)


In [9]:
y[-1]

array([0., 1.], dtype=float32)

In [10]:
# myInput=layers.Input(shape=(height,width,1))
# conv1=Conv2D(16,3, activation='relu',padding='same')(myInput)
# max1=MaxPooling2D(2)(conv1)
# conv2=Conv2D(16,3, activation='relu',padding='same')(max1)
# max2=MaxPooling2D(2)(conv2)
# flat=Flatten()(max2)
# dens1 =Dense(50, activation='relu')(flat)
# dens2 =Dense(10, activation='relu')(dens1)
# out_layer =Dense(1, activation='sigmoid')(dens2)
# model_CNN = Model(myInput, out_layer)
# model_CNN.summary()

In [11]:
#simpler model
# myInput=layers.Input(shape=(height,width,1))
# conv1=Conv2D(8,3, activation='relu',padding='same')(myInput)
# max1=MaxPooling2D(2)(conv1)
# conv2=Conv2D(8,3, activation='relu',padding='same')(max1)
# max2=MaxPooling2D(2)(conv2)
# flat=Flatten()(max2)
# dens1 =Dense(10, activation='relu')(flat)

# out_layer =Dense(2, activation='softmax')(dens1)
# model_CNN = Model(myInput, out_layer)
# model_CNN.summary()

#First Model with accuracy 94%

In [12]:
from keras.backend import dropout
myInput=layers.Input(shape=(number_of_samples,1))
conv1=Conv1D(64,16, activation='relu',padding='same')(myInput)
max1=MaxPooling1D(16)(conv1)
drop1 =Dropout(.2)(max1)
conv2=Conv1D(64,16, activation='relu',padding='same')(drop1)
max2=MaxPooling1D(16)(conv2)
drop2 = Dropout(.2)(max2)
flat=Flatten()(drop2)
dens1 =Dense(100, activation='relu')(flat)
dens2 =Dense(25, activation='relu')(dens1)
out_layer =Dense(2, activation='softmax')(dens2)
model_CNN = Model(myInput, out_layer)
model_CNN.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 160000, 1)]       0         
                                                                 
 conv1d (Conv1D)             (None, 160000, 64)        1088      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 10000, 64)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 10000, 64)         0         
                                                                 
 conv1d_1 (Conv1D)           (None, 10000, 64)         65600     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 625, 64)          0         
 1D)                                                         

In [13]:
# from keras.backend import dropout
# myInput=layers.Input(shape=(number_of_samples,1))
# conv1=Conv1D(64,32, activation='relu',padding='same')(myInput)
# max1=MaxPooling1D(32)(conv1)
# drop1 =Dropout(.2)(max1)
# conv2=Conv1D(64,32, activation='relu',padding='same')(drop1)
# max2=MaxPooling1D(32)(conv2)
# drop2 = Dropout(.2)(max2)
# flat=Flatten()(drop2)
# dens1 =Dense(100, activation='relu')(flat)
# dens2 =Dense(25, activation='relu')(dens1)
# out_layer =Dense(2, activation='softmax')(dens2)
# model_CNN = Model(myInput, out_layer)
# model_CNN.summary()

In [14]:
model_CNN.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics = ['accuracy'])

In [15]:
#train test split
X_train,X_valid,y_train, y_valid = train_test_split(X,y,test_size=.2)

In [16]:
#whether shuffling works
# a = y_test
# noone = 0
# for i in range(a.shape[0]):
#   if a[i][0] == 1 :
#     noone += 1
# print(noone/a.shape[0])

In [ ]:
verbose, epochs, batch_size = 1, 20, 32
# fit network
history = model_CNN.fit(X_train,y_train, epochs=epochs, batch_size=batch_size,validation_data= (X_valid,y_valid))

Epoch 1/20
59/59 [==============================] - 29s 299ms/step - loss: 0.6774 - accuracy: 0.5333 - val_loss: 0.6288 - val_accuracy: 0.6468
Epoch 2/20
59/59 [==============================] - 16s 273ms/step - loss: 0.5910 - accuracy: 0.6424 - val_loss: 0.6701 - val_accuracy: 0.5447
Epoch 3/20
59/59 [==============================] - 16s 273ms/step - loss: 0.5618 - accuracy: 0.6551 - val_loss: 0.5257 - val_accuracy: 0.6936
Epoch 4/20
59/59 [==============================] - 16s 273ms/step - loss: 0.5142 - accuracy: 0.6982 - val_loss: 0.5104 - val_accuracy: 0.7362
Epoch 5/20
59/59 [==============================] - 16s 274ms/step - loss: 0.4846 - accuracy: 0.7653 - val_loss: 0.4765 - val_accuracy: 0.7447
Epoch 6/20
59/59 [==============================] - 16s 274ms/step - loss: 0.4279 - accuracy: 0.8079 - val_loss: 0.4680 - val_accuracy: 0.8106
Epoch 7/20
59/59 [==============================] - 15s 258ms/step - loss: 0.4013 - accuracy: 0.8276 - val_loss: 0.3884 - val_accuracy: 0.8170

In [ ]:
model_CNN.evaluate(X_test,y_test)

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
loss_values = history_dict["loss"]
val_loss_values = history_dict["val_loss"]
epochs = range(1,len(loss_values)+1)
plt.plot(epochs,loss_values,"bo",label = "Training loss")
plt.plot(epochs,val_loss_values,"b", label = "Validation loss")
plt.title("Training and Validation loss")
plt.xlabel("Epochs")
plt.ylabel("loss")
plt.legend()
plt.show()

In [ ]:
acc_values = history_dict["accuracy"]
val_acc_values = history_dict["val_accuracy"]
epochs = range(1,len(acc_values)+1)
plt.plot(epochs,acc_values,"bo",label = "Training accuracy")
plt.plot(epochs,val_acc_values,"b", label = "Validation accuracy")
plt.title("Training and Validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()